## Fixing the issue with losses not matching in 2 codes


In [1]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

import subprocess as sp
import sys
import os
import glob
import pickle 

from matplotlib.colors import LogNorm, PowerNorm, Normalize

import pandas as pd

import argparse
import os
import random


import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

# from torch.utils.data import DataLoader, TensorDataset

import time
from datetime import datetime
import glob
import pickle
import yaml

from scipy import fftpack


In [2]:
%matplotlib widget

In [3]:
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torchsummary import summary
# import torchvision.datasets as dset
# import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np

In [4]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/cosmogan_pytorch/cosmogan/1_main_code/')
import spec_loss as spc
import post_analysis_pandas as post

In [5]:
ip_fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/128_square/dataset_2_smoothing_200k/norm_1_train_val.npy'
img=np.load(ip_fname)[:1000].transpose(0,1,2,3)
t_img=torch.from_numpy(img)
img.shape,t_img.shape

((1000, 1, 128, 128), torch.Size([1000, 1, 128, 128]))

In [8]:
height,width=img.shape[-2:]
height,width

(128, 128)

In [99]:
def f_get_azimuthalAverage(image, center=None):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    source: https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/
    """
    channel, height, width = image.shape
    # Create a grid of points with x and y coordinates
    y, x = np.indices([height,width])
    
    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])

    # Get the radial coordinate for every grid point. Array has the shape of image
    r = np.hypot(x - center[0], y - center[1])

    ind = np.argsort(r.flat) ### Get indices that sort the "r" array in ascending order.

    r_sorted = r.flat[ind]     ### Sort the "r" array
    
    i_sorted = image.flat[ind]   ### Sort the image points according to the radial coordinate

    # Get the integer part of the radii (bin size = 1)
    r_int = r_sorted.astype(int)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = np.where(deltar)[0]       # location of changed radius
    nr = rind[1:] - rind[:-1]        # number of radius bin
    
    # Cumulative sum to figure out sums for each radius bin
    csim = np.cumsum(i_sorted, dtype=float)
    tbin = csim[rind[1:]] - csim[rind[:-1]]

    radial_prof = tbin / nr

    return radial_prof

In [171]:
def f_torch_get_azimuthalAverage_with_batch(image, center=None):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    source: https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/
    """
    
    batch, channel, height, width = image.shape
    # Create a grid of points with x and y coordinates
    y, x = np.indices([height,width])

    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])

    # Get the radial coordinate for every grid point. Array has the shape of image
    r = torch.tensor(np.hypot(x - center[0], y - center[1]))

    # Get sorted radii
    ind = torch.argsort(torch.reshape(r, (batch, channel,-1)))
    r_sorted = torch.gather(torch.reshape(r, (batch, channel, -1,)),2, ind)
    i_sorted = torch.gather(torch.reshape(image, (batch, channel, -1,)),2, ind)

    # Get the integer part of the radii (bin size = 1)
    r_int=r_sorted.to(torch.int32)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[:,:,1:] - r_int[:,:,:-1]  # Assumes all radii represented
    rind = torch.reshape(torch.where(deltar)[2], (batch, -1))    # location of changes in radius
    rind=torch.unsqueeze(rind,1)
    nr = (rind[:,:,1:] - rind[:,:,:-1]).type(torch.float)       # number of radius bin

    # Cumulative sum to figure out sums for each radius bin

    csum = torch.cumsum(i_sorted, axis=-1)
    print(csum.shape,rind.shape,nr.shape)

    tbin = torch.gather(csum, 2, rind[:,:,1:]) - torch.gather(csum, 2, rind[:,:,:-1])
    radial_prof = tbin / nr

    return radial_prof


def f_torch_get_azimuthalAverage(image, center=None):
    """
    Calculate the azimuthally averaged radial profile.

    image - The 2D image
    center - The [x,y] pixel coordinates used as the center. The default is 
             None, which then uses the center of the image (including 
             fracitonal pixels).
    source: https://www.astrobetter.com/blog/2010/03/03/fourier-transforms-of-images-in-python/
    """
    
    height, width = image.shape
    # Create a grid of points with x and y coordinates
    y, x = np.indices([height,width])

    if not center:
        center = np.array([(x.max()-x.min())/2.0, (y.max()-y.min())/2.0])

    # Get the radial coordinate for every grid point. Array has the shape of image
    r = torch.tensor(np.hypot(x - center[0], y - center[1]))

    # Get sorted radii
    ind = torch.argsort(torch.reshape(r, (-1,)))
    r_sorted = torch.gather(torch.reshape(r, ( -1,)),0, ind)
    i_sorted = torch.gather(torch.reshape(image, ( -1,)),0, ind)

    # Get the integer part of the radii (bin size = 1)
    r_int=r_sorted.to(torch.int32)

    # Find all pixels that fall within each radial bin.
    deltar = r_int[1:] - r_int[:-1]  # Assumes all radii represented
    rind = torch.reshape(torch.where(deltar)[0], (-1,))    # location of changes in radius
    nr = (rind[1:] - rind[:-1]).type(torch.float)       # number of radius bin

    # Cumulative sum to figure out sums for each radius bin

    csum = torch.cumsum(i_sorted, axis=-1)
    tbin = torch.gather(csum, 0, rind[1:]) - torch.gather(csum, 0, rind[:-1])
    radial_prof = tbin / nr

    return radial_prof



In [172]:
ans_torch=f_torch_get_azimuthalAverage(t_img[4,0,:,:])

In [100]:
# f_torch_get_azimuthalAverage_with_batch(t_img[4,:,:,:])

ValueError: not enough values to unpack (expected 4, got 3)

In [107]:
ans_numpy=f_get_azimuthalAverage(img[4,:,:,:])

In [119]:
for i in range(ans_numpy.shape[0]):
    a,b=ans_torch[0,i].numpy(),ans_numpy[i]
#     print(a,b)
    if (np.around(a,4)-np.around(b,4)>1e-4):
        print(i,a,b)

In [166]:
t_img.shape,ans_numpy.shape,ans_torch.shape

(torch.Size([1000, 1, 128, 128]), (88,), torch.Size([88]))

In [168]:
ans_torch,ans_numpy

(tensor([-0.8818, -0.8509, -0.7651, -0.7175, -0.6610, -0.6200, -0.7058, -0.7307,
         -0.7219, -0.6810, -0.7214, -0.7372, -0.7409, -0.7078, -0.6914, -0.6791,
         -0.6965, -0.6789, -0.7007, -0.6639, -0.5953, -0.6014, -0.6526, -0.6779,
         -0.7132, -0.7611, -0.7752, -0.7931, -0.7643, -0.7670, -0.7665, -0.7709,
         -0.7416, -0.7432, -0.7680, -0.7805, -0.7790, -0.7858, -0.7713, -0.7417,
         -0.7357, -0.7360, -0.7490, -0.7813, -0.7864, -0.7880, -0.7626, -0.7332,
         -0.7059, -0.7314, -0.7580, -0.7679, -0.7903, -0.7861, -0.7644, -0.7473,
         -0.7007, -0.6825, -0.6788, -0.6964, -0.7341, -0.7191, -0.7534, -0.7738,
         -0.7574, -0.7201, -0.7186, -0.7235, -0.7397, -0.7337, -0.7142, -0.7072,
         -0.7056, -0.6642, -0.6480, -0.6505, -0.6692, -0.6775, -0.6213, -0.6602,
         -0.6875, -0.6728, -0.6850, -0.7434, -0.7876, -0.8026, -0.7846, -0.7190]),
 array([-0.88175743, -0.85091213, -0.76513645, -0.71747654, -0.66103945,
        -0.62000012, -0.70582073, 